In [1]:
import ee
import geemap
import os
import numpy as np
from scipy import signal as sp
from matplotlib import pyplot as plt

In [2]:
manaus_shp = '/Users/morgansteckler/Desktop/tile_shapefiles/venus_fgmanaus_footprint/venus_fgmanaus_footprint.shp'
manaus = geemap.shp_to_ee(manaus_shp)
info = manaus.geometry()
manaus = ee.Geometry(info)

In [8]:
Map = geemap.Map()
Map.centerObject(manaus, 10)
Map

Map(center=[-2.4396454678431994, -59.78996156231432], controls=(WidgetControl(options=['position'], widget=HBo…

In [9]:
#Load ERA 5 mean air temp daily data (.25 arc degree res)
era = (ee.ImageCollection("ECMWF/ERA5/DAILY")
       .select(['mean_2m_air_temperature']) #select temp
       .map(lambda image: image.clip(manaus))
       .filterDate(ee.Date('2016-01-01'), ee.Date('2019-12-31')))

In [11]:
times = era.aggregate_array('system:time_start').getInfo()
print(len(times))
#Note: images appear to be daily (365*4=1460)

1460


In [10]:
#Visualize coarseness on map
Map.addLayer(era, {'min': 295, 'max': 300}, "era temp")

In [11]:
#Loead ERA 5 total precip daily data
era_precip = (ee.ImageCollection("ECMWF/ERA5/DAILY")
       .select(['total_precipitation'])
       .map(lambda image: image.clip(manaus))
       .filterDate(ee.Date('2016-01-01'), ee.Date('2019-12-31')))

In [12]:
#Visualize
Map.addLayer(era_precip, {'min': 0, 'max': .04}, "era precip")

In [ ]:
#Try out GSMaP collection to see how the precip data differs (.1 arc degree res)
gsmap = ee.ImageCollection("JAXA/GPM_L3/GSMaP/v6/operational")